In [1]:
from iterative_retrieval import IterativeRetrieval
rag = IterativeRetrieval("../context/knowledge_base")

Loading transformer model and tokenizer from transformers library: avsolatorio/NoInstruct-small-Embedding-v0
Please wait...

Loading transformer model and tokenizer from transformers library: meta-llama/Meta-Llama-3.1-8B-Instruct
Please wait...



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Combine sub-questions into a multi-hop question

In [2]:
from model_prompts import Prompt

In [ ]:
rag.retriever.get_context("Who was Achilles?")[0]

In [9]:
rag.answer_multi_hop_question("How do you calculate mean?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Decomposing question: How do you calculate mean?


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Extracted sub-question: What is a set of numbers that you would like to find the mean of?
Attempting to answer sub-question...
Attempt 1 to answer question
Retrieved context from article: Arithmetic mean
Attempting to answer question using context: In mathematics and statistics, the arithmetic mean ( ), arithmetic average, or just the "mean" or "average" (when the context is clear) is the sum of a collection of numbers divided by the count of numbers in the collection. The collection is often a set of results from an experiment, an observational study, or a survey. The term "arithmetic mean" is preferred in some mathematics and statistics contexts because it helps distinguish it from other types of means, such as geometric and harmonic.
Answer: I don't know.
Answer confidence:tensor([-0.0010], dtype=torch.float16)


([{'role': 'system',
   'content': "\n    ## Question Decomposition Specialist:\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples:\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The director of Wearing

In [19]:
Prompt.count_number_of_hops = '''
The user needs you to answer a question for them.
They may ask you a simple question which can be answered with one fact, or a complex question which requires many facts to answer.
You must count how many facts the user's question needs to be answered successfully.

## Contraints:
Do not directly answer the user's question, only identify all facts the answer needs.
Write one sentence for each fact.

## Examples:
User: Was Windows XP released in a leap year?
You:
    Fact 1: What year Windows XP was released.
    Fact 2: Whether that year was a leap year.
    Facts needed: 2

User: Who is the president of America?
You:
    Fact 1: Who the current president of America is.
    Facts needed: 1

User: What is the motto of the high school that the creator of Rat Fink attended?
You:
    Fact 1: Who the creator of Rat Fink is.
    Fact 2: What year the creator of Rat Fink attended high school.
    Fact 3: What high school the creator of Rat Fink attended.
    Fact 4: What the motto of that high school was during that year.
    Facts needed: 4

User: Who is the maternal grandfather of Antiochus X Eusebes?
You:
    Fact 1: Who the mother of Antiochus X Eusebes was.
    Fact 2: Who the father of that mother was.
    Facts needed: 2

User: What are the list of prime numbers from 0 to 100?
You:
    Fact 1: The list of prime numbers from 0 to 100.
    Facts needed: 1
'''

In [ ]:
from model_prompts import Prompt

question = "What is Albedo?"

chat_history = [
    {'role':"system",'content':Prompt.count_number_of_hops},
    {'role':'user','content':f"Tell me how many facts I need to answer this question: {question}"},
    {'role':'assistant','content':"Step 1: "}
]

chat_history, num_hops = rag.qd.generate_response(chat_history, max_new_tokens=200)

In [20]:
from model_prompts import Prompt

question = "Who is taller, Abraham Lincoln or Muhammad Ali?"

chat_history = [
    {'role':"system",'content':Prompt.count_number_of_hops},
    {'role':'user','content':f"{question}"},
    {'role':'assistant','content':"Step 1: "}
]

chat_history, num_hops = rag.qd.generate_response(chat_history, max_new_tokens=200)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [21]:
print(num_hops)

Fact 1: Abraham Lincoln's height.
Fact 2: Muhammad Ali's height.
Fact 3: A comparison of their heights to determine who is taller.
Facts needed: 3


In [18]:
print(num_hops)

## Facts Needed to Answer the Question

### Fact 1: Height of Abraham Lincoln
To compare the heights of Abraham Lincoln and Muhammad Ali, we need to know the height of Abraham Lincoln.

### Fact 2: Height of Muhammad Ali
We also need to know the height of Muhammad Ali to make a comparison.

### Fact 3: Comparison of Heights
To answer the question, we need to compare the heights of Abraham Lincoln and Muhammad Ali.

### Fact 4: Height Comparison Result
Finally, we need to know the result of the comparison to answer the question.

## Facts Needed: 4


In [10]:
rag.answer_multi_hop_question("How do you calculate mean?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Decomposing question: How do you calculate mean?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Extracted sub-question: What is the definition of mean?
Attempting to answer sub-question...
Attempt 1 to answer question
Retrieved context from article: Arithmetic mean
Attempting to answer question using context: In mathematics and statistics, the arithmetic mean ( ), arithmetic average, or just the "mean" or "average" (when the context is clear) is the sum of a collection of numbers divided by the count of numbers in the collection. The collection is often a set of results from an experiment, an observational study, or a survey. The term "arithmetic mean" is preferred in some mathematics and statistics contexts because it helps distinguish it from other types of means, such as geometric and harmonic.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: The mean is the sum of a collection of numbers divided by the count of numbers in the collection.
Answer confidence:tensor([0.3096], dtype=torch.float16)
Extracted answer to sub-question: The mean is the sum of a collection of numbers divided by the count of numbers in the collection.
Decomposing question again...


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Evaluating whether contexts are sufficient to answer original query...
Model is confident that it can answer the original question
Retrieved enough context to answer original question: How do you calculate mean?
Context:
['The mean is the sum of a collection of numbers divided by the count of numbers in the collection.']


([{'role': 'system',
   'content': "\n    ## Question Decomposition Specialist:\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples:\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The director of Wearing

In [8]:
rag.retriever.get_context("How do you calculate mean?")[0]

'The arithmetic mean of a set of observed data is equal to the sum of the numerical values of each observation, divided by the total number of observations. Symbolically, for a data set consisting of the values formula_1, the arithmetic mean is defined by the formula:'

In [4]:
rag.retriever.get_context("Who was Achilles?")[0]

'In Greek mythology, Achilles ( ) or Achilleus () was a hero of the Trojan War who was known as being the greatest of all the Greek warriors. The central character in Homer\'s "Iliad", he was the son of the Nereid Thetis and Peleus, king of Phthia and famous Argonaut. Achilles was raised in Phthia along with his childhood companion Patroclus and received his education by the centaur Chiron. In the "Iliad", he is presented as the commander of the mythical tribe of the Myrmidons.'

In [ ]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809."
]

answer = rag.qd.answer_question_using_context(query, contexts, use_chain_of_thought=True)
answer

In [3]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809.",
    "Work on the ASCII standard began in May 1961."
]

answer = rag.qd.answer_question_using_context(query, contexts, use_chain_of_thought=True, max_tokens=300)
answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"The context shows that Abrahan Lincoln was born in the year 1809, whilst the ASCII standard began in 1961. To find the difference, we need to subtract the birth year from the invention year. 1961 - 1809 = 152, therefore 152 years elapsed between Abraham Lincoln's birth and the creation of the ASCII standard."

In [4]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809.",
    "Work on the ASCII standard began in May 1961."
]

is_answer_attainable, chat_history = rag.is_answer_attainable(query, contexts)

print(f"Is answer attainable (should be True): {is_answer_attainable}")
chat_history

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Is answer attainable (should be True): False


[{'role': 'system',
  'content': '\n    ## Answer Deduction Specialist.\n    You are an expert at telling whether a question needs follow-up questions to answer or not.\n    The user will give you a question and background contexts. Based on the contexts and the question, deduce if the information provided is sufficient to answer the question, or whether follow-up questions are needed to find the answer.\n    \n    ## Constraints:\n    Your answer must be "Yes" if follow-up questions are needed or "No" if the question can be answered from the inputs.\n    \n    ## Examples:\n    Question: Who is the maternal grandfather of Antiochus X Eusebes?\n    Context: The mother of Antiochus X Eusebes is Cleopatra IV.\n    Context: The father of Cleopatra IV is Ptolemy VIII Physcon.\n    Are follow up questions needed here: No.\n\n    Question: Steven Spielberg is from the United States.\n    Context: Are both the directors of Jaws and Casino Royale from the same country?\n    Context: The direct

In [5]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809."
]

is_answer_attainable, chat_history = rag.is_answer_attainable(query, contexts)

print(f"Is answer attainable (should be False): {is_answer_attainable}")
chat_history

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Is answer attainable (should be False): False


[{'role': 'system',
  'content': '\n    ## Answer Deduction Specialist.\n    You are an expert at telling whether a question needs follow-up questions to answer or not.\n    The user will give you a question and background contexts. Based on the contexts and the question, deduce if the information provided is sufficient to answer the question, or whether follow-up questions are needed to find the answer.\n    \n    ## Constraints:\n    Your answer must be "Yes" if follow-up questions are needed or "No" if the question can be answered from the inputs.\n    \n    ## Examples:\n    Question: Who is the maternal grandfather of Antiochus X Eusebes?\n    Context: The mother of Antiochus X Eusebes is Cleopatra IV.\n    Context: The father of Cleopatra IV is Ptolemy VIII Physcon.\n    Are follow up questions needed here: No.\n\n    Question: Steven Spielberg is from the United States.\n    Context: Are both the directors of Jaws and Casino Royale from the same country?\n    Context: The direct

In [6]:
is_answer_attainable

False

In [7]:
query = "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?"
contexts = [
    "Abraham Lincoln was born on February 12, 1809.",
    "Work on the ASCII standard began in May 1961."
]

is_answer_attainable = rag.is_answer_attainable(query, contexts)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [6]:
bool(is_answer_attainable[0])

True

In [11]:
chat_history, sub_question = rag.qd.decompose_question_step(chat_history)

print(sub_question)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


When was the ASCII standard completed and officially published?


In [9]:
a = [{"a"}, {"b"}]

b = [{"c"}, *a]

b

[{'c'}, {'a'}, {'b'}]

In [ ]:
chat_history = rag.answer_multi_hop_question("How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?")

print(chat_history)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Decomposing question: How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?


c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Extracted sub-question: To determine the number of years that lapsed between Abraham Lincoln's birth and the invention of ASCII, I'll need to know a few things first. What year was Abraham Lincoln born in?
Attempting to answer sub-question...
Attempt 1 to answer question


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Retrieved context from article: Abraham Lincoln
Attempting to answer question using context: Lincoln was largely self-educated. His formal schooling was from itinerant teachers. It included two short stints in Kentucky, where he learned to read, but probably not to write. In Indiana at age seven, due to farm chores, he attended school only sporadically, for a total of fewer than 12 months in aggregate by age 15. Nonetheless, he remained an avid reader and retained a lifelong interest in learning. Family, neighbors, and schoolmates recalled that his readings included the King James Bible, Aesop's Fables, John Bunyan's "The Pilgrim's Progress", Daniel Defoe's "Robinson Crusoe", and "The Autobiography of Benjamin Franklin". Despite being self-educated, Lincoln was the recipient of honorary degrees later in life, including an honorary Doctor of Laws from Columbia University in June 1861.


In [4]:
final_answer = rag.qd.answer_question_using_context(
    "How many years lapsed between Abraham Lincoln's birth and the invention of ASCII?",
    " ".join([
        "Abraham Lincoln was born on February 12, 1809.",
        "Work on the ASCII standard began in May 1961."
    ])
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [5]:
final_answer

"There are 52 years between Abraham Lincoln's birth and the invention of ASCII."

In [ ]:
chat_history = [
    {'role':'system','content':}
]